In [9]:
import numpy as np
import json
import matplotlib.pyplot as plt
from qutip import qeye, destroy, tensor
from qutip.core.gates import *
from qutip import Qobj
from qutip import *  

pi = np.pi

# -----------------------------------------------------------
# Utility functions
# -----------------------------------------------------------

def wc_t(t, args=None):
    """Initial time, no cavity coupling yet """
    return 1 if t <= T0_1 else 0


def w1_t(t, args=None):
    """Control envelope for qubit 1."""
    return 1 if T0_1 < t <= T0_1 + T_gate_1 else 0


def w2_t(t, args=None):
    """Control envelope for qubit 2."""
    return 1 if T0_2 < t <= T0_2 + T_gate_2 else 0


def read_json_to_dict(filename: str) -> dict:
    """Read a JSON file into a Python dictionary."""
    with open(filename, "r") as f:
        return json.load(f)


def _tensor(*ops):
    """Kronecker product of multiple operators (NumPy arrays)."""
    result = ops[0]
    for op in ops[1:]:
        result = np.kron(result, op)
    return result


def basis_state(dim, index):
    """Return computational basis vector |index> in a Hilbert space of dimension dim."""
    state = np.zeros((dim, 1))
    state[index, 0] = 1
    return state


def build_projector(dim, index_a, index_b):
    """Return projector |a><b| for given basis indices in a Hilbert space of dimension dim."""
    state_a = basis_state(dim, index_a)
    state_b = basis_state(dim, index_b)
    return state_a @ state_b.T.conj()


def build_qubit_cavity_vibronic_coupling(
    H_qubit_cavity_coupling, omega_list, gv_list, boson_dim, qubit_number=1
):
    """
    Build Pauli–Fierz Hamiltonian for qubit–cavity coupling and vibronic interaction.

    Args:
        H_qubit: Qubit Hamiltonian (Qobj).
        H_cavity: Cavity Hamiltonian (Qobj).
        d_matrix: Dipole matrix for qubit.
        omega_cavity: Cavity frequency.
        omega_list: List of vibrational frequencies.
        gv_list: List of coupling strengths.
        boson_dim: Dimension of bosonic Hilbert space.
        qubit_number: Which qubit couples (1 or 2).

    Returns:
        Vibronic coupling Hamiltonian in the uncoupled basis (NumPy array).
    """
    # Identities
    Iq = qeye(2)
    Iq_np = np.eye(2)
    Ic = qeye(boson_dim)
    Iv = qeye(boson_dim)
    Ic_np = np.eye(boson_dim)
    Iv_np = np.eye(boson_dim)

    # Bosonic operators
    am = destroy(boson_dim)
    ap = am.dag()
    nc = ap * am
    am_np, ap_np, nc_np = am.full(), ap.full(), nc.full()



    # Diagonalize the qubit-cavity coupled Hamiltonian (e.g. Pauli-Fierz, Rabi, JC, depending on what is supplied as input)
    H_qubit_cavity_coupling_numpy = H_qubit_cavity_coupling.full()
    vals, vecs = np.linalg.eigh(H_qubit_cavity_coupling_numpy)

    # Number of polariton states
    N_pol = 2 * boson_dim

    # Vibronic Hamiltonian (allocate properly as matrices!)
    dim_total = N_pol * 2 * boson_dim * boson_dim
    H_pol_vib = np.zeros((dim_total, dim_total), dtype=complex)
    H_pol_vib_coup = np.zeros((dim_total, dim_total), dtype=complex)

    for i in range(N_pol):
        Proj_ii = build_projector(N_pol, i, i)

        if qubit_number == 1:
            H_pol_vib += omega_list[i] * _tensor(Proj_ii, Iq_np, nc_np, Iv_np)
        else:
            H_pol_vib += omega_list[i] * _tensor(Iq_np, Proj_ii, Iv_np, nc_np)

        # consider vibronic coupling between state 0 and all others
        if i>0:
            Proj_0i = build_projector(N_pol, 0, i) + build_projector(N_pol, i, 0)

            if qubit_number == 1:
                H_pol_vib_coup += gv_list[i] * _tensor(Proj_0i, Iq_np, am_np + ap_np, Iv_np)
            else:
                H_pol_vib_coup += gv_list[i] * _tensor(Iq_np, Proj_0i, Iv_np, am_np + ap_np)

    # Transform back to uncoupled basis
    H_vib_coup = vecs.T.conj() @ (H_pol_vib + H_pol_vib_coup) @ vecs

    qubit_dims = [2, boson_dim, 2, boson_dim, boson_dim]
    return Qobj(H_vib_coup, dims=[qubit_dims, qubit_dims])

                
# -----------------------------------------------------------
# Load system parameters
# -----------------------------------------------------------
LiH_params = read_json_to_dict("LiH_params.json")

Nf = 3   # bosonic cutoff
# local operators
sz = sigmaz()
sm = destroy(2)
sp = sm.dag()
nq = sp * sm

am = destroy(Nf)
ap = am.dag()
nc = ap * am

# identities
Iq = qeye(2)
Ic = qeye(Nf)
Iv = qeye(Nf)

# fixed frequencies
omega_q = LiH_params["w_q1"]
omega_c = omega_q  # enforce resonance
omega_v = LiH_params["w_vib1"]
mu_eg = LiH_params["qubit_1_dipole_moments"]["mu_eg"]
mu_ee = LiH_params["qubit_1_dipole_moments"]["mu_e"]
mu_gg = LiH_params["qubit_1_dipole_moments"]["mu_g"]

# define list of omega values for vibrational frequencies of polariton states, there should be 2 * Nf of these values
# for Nf = 3, these are ordered as frequency of coupled gs, frequency of LP, frequency of UP, frequency of S0, frequency of S1, frequency of S1 
omega_vib_list = [0.006, # omega on gs
                  0.006, # omega on LP
                  0.006, # omega on UP
                  0.006, # omega on S0
                  0.006, # omega on S1
                  0.006] # omega on S1 

# gv values for transitions between grouna and and excited polariton states
# for Nf = 3, these are ordered as gv for coupled gs to LP, coupled gs to UP, coupled gs to S0 (0), coupled gs to S1, coupled gs to S1
gv_list = [0.0, # not used
           0.0, # gs to LP
           0.0, # gs to UP
           0.0, # gs to S0
           0.0, # gs to S1
           0.0] # gs to S1           


# state ordering is q1, cav, q2, v1, v2

# initial state: |e,0,g,0,0>
psi0 = tensor(basis(2,1), basis(Nf,0),
              basis(2,0), basis(Nf,0), basis(Nf,0))

# ideal Bell target
rho_qubits_ideal = ket2dm(
    tensor(phasegate(0), phasegate(pi/2)) *
    sqrtiswap() *
    tensor(basis(2,1), basis(2,0))
)

# time discretization
tlist = np.linspace(0, 1400, 5000)

# bare Hamiltonian (independent of gc, gv)
H_q1 = tensor(-omega_q/2 * sz, Ic, Iq, Iv, Iv)
H_cav = tensor(Iq, omega_c * nc, Iq, Iv, Iv)
H_q2 = tensor(Iq, Ic,  -omega_q/2 * sz, Iv, Iv)
H_v1 = tensor(Iq, Ic, Iq, omega_v * nc, Iv)
H_v2 = tensor(Iq, Ic, Iq, Iv, omega_v * nc)
H_q1_v1 = gv_list[4] * tensor(sp * sm, Ic, Iq, (am + ap), Iv)
H_q2_v2 = gv_list[4] * tensor(Iq, Ic, sp * sm, Iv, (am + ap))



# -----------------------------------------------------------
# Parameter scan
# -----------------------------------------------------------
lv = 0.05
gc_fixed = np.sqrt( omega_c / 2) * lv * np.abs(mu_eg)  # <-- gc_fixed determined by lv, omega_c, and mu_eg

# dipole operator (fixed by lv) - used in Rabi and PF couplings
d_matrix_fixed = lv * np.array([[mu_gg, mu_eg], [mu_eg, mu_ee]])
d_qobj = Qobj(d_matrix_fixed)


# Rabi bilinear coupling
H_q1_cav_Rabi = np.sqrt(omega_c / 2) * tensor(d_qobj, (am + ap), Iq, Iv, Iv)
H_q2_cav_Rabi = np.sqrt(omega_c / 2) * tensor(Iq, (am + ap), d_qobj, Iv, Iv, )

# Pauli-Fierz is Rabi + DSE
H_q1_cav_PF = H_q1_cav_Rabi + 1/2 * tensor(d_qobj * d_qobj, Ic, Iq, Iv, Iv)
H_q2_cav_PF = H_q2_cav_Rabi + 1/2 * tensor(Iq, Ic, d_qobj * d_qobj, Iv, Iv)

# get the coupling between qubit1 and cavity and vibrations based on PF Hamiltonian and the omega_vib_list and gv_list defined above
H_q1_cav_PF_vib = build_qubit_cavity_vibronic_coupling(
    H_q1_cav_PF, omega_vib_list, gv_list, Nf, qubit_number=1
)

# get the coupling between qubit2 and cavity and vibrations based on PF Hamiltonian and the omega_vib_list and gv_list defined above
H_q2_cav_PF_vib = build_qubit_cavity_vibronic_coupling(
    H_q2_cav_PF, omega_vib_list, gv_list, Nf, qubit_number=2
)   


# gate times fixed by gc
global T0_1, T0_2, T_gate_1, T_gate_2
T0_1 = 20
T_gate_1 = 338.91 #pi / (4 * np.abs(gc_fixed))
T0_2 = T0_1 + T_gate_1
T_gate_2 =  128.32 #pi / (2 * np.abs(gc_fixed))

H_uncoupled = H_q1 + H_cav + H_q2 + H_v1 + H_v2 + H_q1_v1 + H_q2_v2
H_q1_cav_coupled = H_q1_cav_PF + H_q1_cav_PF_vib + H_q2 + H_v2 + H_q2_v2
H_q2_cav_coupled = H_q2_cav_PF + H_q2_cav_PF_vib + H_q1 + H_v1 + H_q1_v1
    
H_t_PF = [[H_uncoupled, wc_t], [H_q1_cav_coupled, w1_t], [H_q2_cav_coupled, w2_t]]


res_PF = mesolve(H_t_PF, psi0, tlist, [], e_ops=[])
rho_final_PF = res_PF.states[-1]



rho_qubits_PF = ptrace(rho_final_PF, [0,2])


print("Fidelity of PF qubit state with ideal Bell state:", fidelity(rho_qubits_PF, rho_qubits_ideal))
print("concurrence of PF qubit state:", concurrence(rho_qubits_PF))


Fidelity of PF qubit state with ideal Bell state: 0.6565345582417286
concurrence of PF qubit state: 0.0


In [ ]:
import matplotlib.ticker as ticker
fig, ax = plt.subplots(figsize=(6,4))
ax.plot(gc_fixed / gv_values, fidelities, 'o-', label="Fidelity")
ax.plot(gc_fixed / gv_values, concurrences, 's-', label="Concurrence")

ax.set_xlabel("gv")
ax.set_ylabel("Value")
ax.set_title(f"Fidelity & Concurrence vs gv (gc fixed = {gc_fixed})")
ax.legend()
ax.grid(True)

# Force scientific notation on x-axis
ax.xaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
ax.ticklabel_format(axis="x", style="sci", scilimits=(0,0))
#ax.xaxis.set_major_formatter(ticker.FormatStrFormatter('%.3e'))
ax.loglog()
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# === Global Styling (applies to all figures) ===
plt.rcParams.update({
    "figure.figsize": (6, 4),          # consistent size
    "font.family": "serif",            # journal-style (use "Times New Roman" if required)
    "font.size": 12,                   # base font size
    "axes.labelsize": 12,              # axis labels
    "axes.titlesize": 13,              # title
    "axes.titleweight": "bold",
    "legend.fontsize": 10,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10,
    "lines.linewidth": 2,              # thicker lines
    "lines.markersize": 6,
    "grid.linestyle": "--",
    "grid.linewidth": 0.6,
    "grid.alpha": 0.7
})

# === Your plot ===
fig, ax = plt.subplots()

ax.plot(gc_fixed / gv_values, fidelities, 'o-', color="C0", label="Fidelity")
ax.plot(gc_fixed / gv_values, concurrences, 's--', color="C1", label="Concurrence")

ax.set_xlabel(r"$g_c / g_v$")
ax.set_ylabel("Value")
ax.set_title(f"Fidelity & Concurrence vs $g_c / g_v$\n($g_c$ fixed = {gc_fixed})")

ax.legend(frameon=True, loc="best")
ax.grid(True, which="both")

# Force scientific notation + log scales
ax.xaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
ax.ticklabel_format(axis="x", style="sci", scilimits=(0,0))
ax.set_xscale("log")
ax.set_yscale("log")
#ax.set_xlim(1,5)
plt.tight_layout()

# === Save as vector graphics for publication ===
plt.savefig("fidelity_concurrence_vs_gv.pdf", bbox_inches="tight")   # vector, scalable
plt.savefig("fidelity_concurrence_vs_gv.png", dpi=600, bbox_inches="tight")  # high-res raster

plt.show()

